In [4]:
# Импорт основных библиотек
import numpy as np
import pandas as pd

# Импорт библиотек построения графиков и диаграмм
from matplotlib import pyplot as plt
import seaborn as sns

# Указание режима отображения диаграмм
%matplotlib inline

# Импорт основных библиотек машинного обучения
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler

# Импорт библиотек моделей машинного обучения
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier

# Импорт библиотек отбора признаков
from sklearn.feature_selection import RFE

# Импорт библиотек оценки моделей машинного обучения
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# Импорт библиотек добавления синтетических данных
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN 

from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from scipy.io.arff import loadarff 

In [12]:
raw_data1 = loadarff('/home/timur/Сlassification_traffic/data/raw/TimeBasedFeatures-Dataset-15s-VPN.arff')
df1 = pd.DataFrame(raw_data1[0])

raw_data2 = loadarff('/home/timur/Сlassification_traffic/data/raw/TimeBasedFeatures-Dataset-30s-VPN.arff')
df2 = pd.DataFrame(raw_data2[0])

raw_data3 = loadarff('/home/timur/Сlassification_traffic/data/raw/TimeBasedFeatures-Dataset-60s-VPN.arff')
df3 = pd.DataFrame(raw_data3[0])

raw_data4 = loadarff('/home/timur/Сlassification_traffic/data/raw/TimeBasedFeatures-Dataset-120s-VPN.arff')
df4 = pd.DataFrame(raw_data4[0])

In [13]:
df1

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,...,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,class1
0,2218398.0,2191528.0,2192676.0,9.0,17.0,60217.0,86113.0,6.010773e+02,370.009450,4315.726934,...,1815.741336,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-BROWSING'
1,3517813.0,3517813.0,3304404.0,6.0,7.0,324059.0,217426.0,1.413912e+03,998.912938,1648.183118,...,8971.721519,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-BROWSING'
2,13567657.0,13394098.0,13437531.0,0.0,0.0,3906748.0,3908013.0,1.060499e+04,5204.310999,283.541956,...,65256.442153,5816127.0,5816127.0,5816127.0,0.000000,3777807.0,3777807.0,3777807.0,0.000000,b'VPN-BROWSING'
3,14697122.0,14567358.0,14567724.0,4.0,14.0,2447126.0,2406746.0,1.579974e+04,6497.646744,215.416324,...,51735.382672,5068736.0,5435233.5,5801731.0,518305.735075,1141447.0,1708300.0,2275153.0,801651.200471,b'VPN-BROWSING'
4,270207.0,203681.0,244006.0,15.0,10.0,27147.0,66572.0,5.938222e+02,363.104167,3763.781101,...,2648.293598,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-BROWSING'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9788,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,0.000000,0.000000,...,0.000000,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-VOIP'
9789,895.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,0.000000,2234.636872,...,0.000000,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-VOIP'
9790,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,0.000000,0.000000,...,0.000000,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-VOIP'
9791,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.000000e+00,0.000000,0.000000,...,0.000000,-1.0,0.0,-1.0,0.000000,-1.0,0.0,-1.0,0.000000,b'VPN-VOIP'


In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9793 entries, 0 to 9792
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   duration            9793 non-null   float64
 1   total_fiat          9793 non-null   float64
 2   total_biat          9793 non-null   float64
 3   min_fiat            9793 non-null   float64
 4   min_biat            9793 non-null   float64
 5   max_fiat            9793 non-null   float64
 6   max_biat            9793 non-null   float64
 7   mean_fiat           9793 non-null   float64
 8   mean_biat           9793 non-null   float64
 9   flowPktsPerSecond   9793 non-null   float64
 10  flowBytesPerSecond  9793 non-null   float64
 11  min_flowiat         9793 non-null   float64
 12  max_flowiat         9793 non-null   float64
 13  mean_flowiat        9793 non-null   float64
 14  std_flowiat         9793 non-null   float64
 15  min_active          9793 non-null   float64
 16  mean_a